In [2]:
import os
import re
import time
import gc
import random
import numpy as np

# Force TensorFlow to run on CPU'

import tensorflow as tf
from tensorflow.keras.applications import VGG19, DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

# Set precision globally
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('float32')

print("Using devices:", tf.config.get_visible_devices('GPU'), tf.config.get_visible_devices('CPU'))

def clear_memory():
    K.clear_session()
    tf.keras.backend.clear_session()
    gc.collect()

# ——— CHECKPOINT UTILITY ———
def get_latest_checkpoint(dir_path):
    files = [f for f in os.listdir(dir_path) if f.startswith("model_checkpoint_lantana_cucoo_epoch_")]
    print("Checkpoints found:", files)
    if not files:
        return None, 0
    epochs = [int(re.search(r'epoch_(\d+)', f).group(1)) for f in files if re.search(r'epoch_(\d+)', f)]
    idx = max(range(len(epochs)), key=lambda i: epochs[i])
    return os.path.join(dir_path, files[idx]), epochs[idx]

# ——— USER SETTINGS ———
selected_optimizer = 'adam'
train_dir      = 'C:/Users/HP/Desktop/lantana_images/lantana_images/train'
validation_dir = 'C:/Users/HP/Desktop/lantana_images/lantana_images/val'
test_dir       = 'C:/Users/HP/Desktop/lantana_images/lantana_images/test'
checkpoint_dir = './checkpoints_lantana_cucoo'
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, 'model_checkpoint_lantana_cucoo_epoch_{epoch:02d}.keras')

# ——— DATA GENERATORS ———
batch_size = 8
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(train_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
val_gen   = datagen.flow_from_directory(validation_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary')
test_gen  = datagen.flow_from_directory(test_dir, target_size=(224,224), batch_size=batch_size, class_mode='binary', shuffle=False)

# ——— MODEL UTILITIES ———
def freeze_all(model):
    for layer in model.layers:
        layer.trainable = False

def unfreeze_layers(model, mode):
    if mode == 'all':
        for layer in model.layers:
            layer.trainable = True
    elif mode == 'last5':
        for layer in model.layers[-5:]:
            layer.trainable = True

def build_model():
    inp = Input(shape=(224,224,3))
    vgg = VGG19(weights='imagenet', include_top=False, input_tensor=inp)
    dn  = DenseNet121(weights='imagenet', include_top=False, input_tensor=inp)
    freeze_all(vgg)
    freeze_all(dn)

    x1 = GlobalAveragePooling2D()(vgg.output)
    x2 = GlobalAveragePooling2D()(dn.output)
    x  = Concatenate()([x1, x2])
    x  = Dense(256, activation='relu')(x)
    out= Dense(1, activation='sigmoid', dtype='float32')(x)
    model = Model(inputs=inp, outputs=out)
    return model

# ——— EVALUATION FUNCTION FOR CSO ———
def evaluate_lr(lr_val):
    lr = float(lr_val[0]) if isinstance(lr_val, (list, np.ndarray)) else float(lr_val)
    model = build_model()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(train_gen, validation_data=val_gen, epochs=1, steps_per_epoch=2, validation_steps=1, verbose=0)
    val_acc = history.history['val_accuracy'][-1]
    clear_memory()
    return -val_acc

class CuckooSearch:
    def __init__(self, objective_function, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=5):
        self.f = objective_function
        self.n = n
        self.pa = pa
        self.alpha = alpha
        self.lb = np.array(lower_bound)
        self.ub = np.array(upper_bound)
        self.max_iter = max_iter
        self.d = len(lower_bound)

    def levy_flight(self):
        from scipy.special import gamma
        beta = 1.5
        sigma = (gamma(1 + beta) * np.sin(np.pi * beta / 2) / (gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)
        u = np.random.randn(self.d) * sigma
        v = np.random.randn(self.d)
        step = u / np.abs(v)**(1 / beta)
        return step

    def search(self):
        nests = np.random.uniform(self.lb, self.ub, size=(self.n, self.d))
        fitness = np.array([self.f(nest) for nest in nests])
        best_idx = np.argmin(fitness)
        best_nest = nests[best_idx].copy()
        best_fitness = fitness[best_idx]

        for t in range(self.max_iter):
            for i in range(self.n):
                step = self.alpha * self.levy_flight()
                new_nest = nests[i] + step * (nests[i] - best_nest)
                new_nest = np.clip(new_nest, self.lb, self.ub)
                new_fit = self.f(new_nest)
                if new_fit < fitness[i]:
                    nests[i] = new_nest
                    fitness[i] = new_fit
                    if new_fit < best_fitness:
                        best_nest = new_nest
                        best_fitness = new_fit

            for i in range(self.n):
                if random.random() < self.pa:
                    nests[i] = np.random.uniform(self.lb, self.ub, self.d)
                    fitness[i] = self.f(nests[i])
                    if fitness[i] < best_fitness:
                        best_nest = nests[i]
                        best_fitness = fitness[i]

            print(f"[CS Iter {t+1}/{self.max_iter}] Best fitness = {-best_fitness:.4f}")

        return best_nest, best_fitness

def run_cuckoo_search():
    print("🔍 Running Cuckoo Search to find best learning rate...")
    cs = CuckooSearch(evaluate_lr, n=5, pa=0.25, alpha=0.01, lower_bound=[1e-6], upper_bound=[1e-3], max_iter=5)
    best_nest, best_score = cs.search()
    best_lr = float(best_nest[0])
    print(f"✅ Best LR found: {best_lr:.6f} with validation accuracy {-best_score:.4f}")
    return best_lr

# ——— OPTIMIZER FACTORY ———
def get_opt(name, lr):
    if name == 'adamw':
        return tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=1e-5)
    elif name == 'nadam':
        return tf.keras.optimizers.Nadam(learning_rate=lr)
    else:
        return tf.keras.optimizers.Adam(learning_rate=lr)

# ——— LOAD FROM CHECKPOINT IF AVAILABLE ———
latest_ckpt, last_epoch = get_latest_checkpoint(checkpoint_dir)
if latest_ckpt and os.path.exists(latest_ckpt):
    model = load_model(latest_ckpt, compile=False)
    print(f"Resuming from epoch {last_epoch}")
    model.compile(optimizer=get_opt(selected_optimizer, 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
else:
    model = build_model()
    last_epoch = 0
    print("Starting fresh training")

ckpt_cb = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1)

# ——— TRAINING PHASES ———
lr_phase_0 = run_cuckoo_search()

phases = [
    (lr_phase_0, 10, []),
    (1e-5, 5, ['last5']),
    (1e-6, 5, ['all']),
]

histories = []

for idx, (lr, epochs, flags) in enumerate(phases):
    phase_start_epoch = sum(p[1] for p in phases[:idx])
    phase_end_epoch = phase_start_epoch + epochs
    if last_epoch >= phase_end_epoch:
        continue

    print(f"🚀 Starting Phase {idx+1}: lr={lr}, epochs={epochs}, flags={flags}")

    if 'all' in flags:
        unfreeze_layers(model, 'all')
    elif 'last5' in flags:
        unfreeze_layers(model, 'last5')

    model.compile(optimizer=get_opt(selected_optimizer, lr), loss='binary_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=phase_end_epoch,
        initial_epoch=last_epoch,
        callbacks=[ckpt_cb],
        verbose=1
    )
    histories.append(history)
    last_epoch = phase_end_epoch

# ——— TRAINING AND VALIDATION ACCURACY, PRECISION, RECALL, F1 SCORE ———
def evaluate_model(model, gen):
    y_true = gen.classes
    y_pred = (model.predict(gen, verbose=1) > 0.5).astype('int32')
    
    accuracy = (y_true == y_pred).mean()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    return accuracy, precision, recall, f1

train_accuracy, train_precision, train_recall, train_f1 = evaluate_model(model, train_gen)
val_accuracy, val_precision, val_recall, val_f1 = evaluate_model(model, val_gen)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1-score: {train_f1:.4f}")
print(val_accuracy)


Using devices: [] [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Found 10872 images belonging to 2 classes.
Found 2330 images belonging to 2 classes.
Found 2330 images belonging to 2 classes.
Checkpoints found: ['model_checkpoint_lantana_cucoo_epoch_01.keras', 'model_checkpoint_lantana_cucoo_epoch_02.keras', 'model_checkpoint_lantana_cucoo_epoch_03.keras', 'model_checkpoint_lantana_cucoo_epoch_04.keras', 'model_checkpoint_lantana_cucoo_epoch_05.keras', 'model_checkpoint_lantana_cucoo_epoch_06.keras', 'model_checkpoint_lantana_cucoo_epoch_07.keras', 'model_checkpoint_lantana_cucoo_epoch_08.keras', 'model_checkpoint_lantana_cucoo_epoch_09.keras', 'model_checkpoint_lantana_cucoo_epoch_10.keras', 'model_checkpoint_lantana_cucoo_epoch_11.keras', 'model_checkpoint_lantana_cucoo_epoch_12.keras', 'model_checkpoint_lantana_cucoo_epoch_13.keras', 'model_checkpoint_lantana_cucoo_epoch_14.keras', 'model_checkpoint_lantana_cucoo_epoch_15.keras']
Resuming from epoch 15
🔍 Running 

In [3]:
print(val_accuracy)

0.4999668441120669
